In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from scipy.optimize import minimize
import os

# =============================================================================
# 1. 파일 경로 설정 (사용자 요청 반영)
# =============================================================================
DATA_PATH = '../../data/raw/'   # train.csv가 위치한 경로 (기존 유지)
NPY_PATH = './oof_data/'        # npy 파일이 위치한 경로 (업데이트됨)

# OOF 파일명 (사용자 보유 파일)
npy_m1 = 'exp11_sota_AUC_0.77212.npy'       # Base
npy_m2 = 'exp12_auc_AUC_0.77233.npy'        # AUC Optimized
npy_m7 = 'exp25_m7_Refined_AUC_0.77375.npy' # Refined FE

# =============================================================================
# 2. 데이터 로드 및 정밀 동기화 (Synchronization)
# =============================================================================
print(">>> 데이터 정렬 및 동기화 작업 시작...")

# (1) 정답지 로드
try:
    train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
except FileNotFoundError:
    print(f"오류: {DATA_PATH}train.csv를 찾을 수 없습니다. 경로를 확인해주세요.")
    raise

full_voted = train['voted'].values

# (2) 필터 마스크 생성
# m1, m2가 사용한 1차 필터 (가족 수 <= 50)
mask_family = train['familysize'] <= 50 

# m7이 사용한 2차 필터 (불성실 응답자 제거)
# m7은 mask_family가 적용된 상태에서 추가로 필터링을 수행했음
a_cols = [col for col in train.columns if col.endswith('A') and col.startswith('Q')]
# 전체 데이터 기준 불성실 응답자(std=0) 마스크
mask_std_global = train[a_cols].std(axis=1) > 0 

# (3) NPY 파일 로드
try:
    p1_raw = np.load(os.path.join(NPY_PATH, npy_m1))
    p2_raw = np.load(os.path.join(NPY_PATH, npy_m2))
    p7_raw = np.load(os.path.join(NPY_PATH, npy_m7))
except FileNotFoundError as e:
    print(f"오류: OOF 파일을 찾을 수 없습니다. {e}")
    raise

# 1차원 펼치기
if p1_raw.ndim > 1: p1_raw = p1_raw.ravel()
if p2_raw.ndim > 1: p2_raw = p2_raw.ravel()
if p7_raw.ndim > 1: p7_raw = p7_raw.ravel()

print(f"   - [Load Raw] m1: {len(p1_raw)}, m2: {len(p2_raw)}, m7: {len(p7_raw)}")

# (4) 데이터 동기화 로직 (Slicing)
# 상황: m1, m2는 mask_family만 적용됨 (45,529)
#       m7은 mask_family AND mask_std_global 적용됨 (45,490)
# 목표: m1, m2에서 "mask_family는 통과했지만 mask_std_global은 통과 못한" 39명을 제거

# step 1: mask_family를 통과한 데이터프레임의 인덱스를 기준으로, mask_std 여부 확인
df_family_filtered = train[mask_family]
# mask_family 통과한 애들 중에서, std > 0 인 애들만 True
mask_final_sync = df_family_filtered[a_cols].std(axis=1) > 0

# step 2: m1, m2 필터링 적용
p1_sync = p1_raw[mask_final_sync.values]
p2_sync = p2_raw[mask_final_sync.values]
p7_sync = p7_raw # m7은 이미 다 적용된 상태

# step 3: 정답지(Target) 필터링
# 전체 -> 가족필터 -> std필터 순으로 적용
y_true_sync = train[mask_family][mask_final_sync]['voted'].values

# (5) 정합성 검증
print("-" * 30)
print(f"   - [Sync] m1: {len(p1_sync)}")
print(f"   - [Sync] m2: {len(p2_sync)}")
print(f"   - [Sync] m7: {len(p7_sync)}")
print(f"   - [Target] y: {len(y_true_sync)}")

if not (len(p1_sync) == len(p2_sync) == len(p7_sync) == len(y_true_sync)):
    print(f"동기화 실패! (m7 원본: {len(p7_raw)} vs 계산된 타겟: {len(y_true_sync)})")
    raise ValueError("데이터 개수가 맞지 않습니다. m7 생성 시점의 전처리와 현재 로직이 다를 수 있습니다.")
else:
    print("데이터 동기화 완료! (N=45,490)")

# =============================================================================
# 3. 최적 비율 탐색 (SLSQP)
# =============================================================================
# 방향성 보정
def check_flip(pred, true):
    score = roc_auc_score(true, pred)
    if score < 0.5: return 1 - pred
    return pred

p1_sync = check_flip(p1_sync, y_true_sync)
p2_sync = check_flip(p2_sync, y_true_sync)
p7_sync = check_flip(p7_sync, y_true_sync)

# 랭크 변환
r1 = pd.Series(p1_sync).rank(pct=True)
r2 = pd.Series(p2_sync).rank(pct=True)
r7 = pd.Series(p7_sync).rank(pct=True)

def objective(weights):
    w1, w2, w7 = weights
    final_rank = (r1 * w1) + (r2 * w2) + (r7 * w7)
    return 1.0 - roc_auc_score(y_true_sync, final_rank)

cons = ({'type': 'eq', 'fun': lambda w: 1 - sum(w)})
bnds = [(0, 1), (0, 1), (0, 1)]
init = [0.33, 0.33, 0.34]

res = minimize(objective, init, method='SLSQP', bounds=bnds, constraints=cons)
best_w = res.x
best_auc = 1 - res.fun

print("-" * 30)
print(f"최적 비율 발견 (Local Max AUC: {best_auc:.6f})")
print(f"   m1 (Base): {best_w[0]*100:.2f}%")
print(f"   m2 (AUC) : {best_w[1]*100:.2f}%")
print(f"   m7 (DFE) : {best_w[2]*100:.2f}%")
print("-" * 30)

>>> 데이터 정렬 및 동기화 작업 시작...
   - [Load Raw] m1: 45529, m2: 45529, m7: 45490
------------------------------
   - [Sync] m1: 45490
   - [Sync] m2: 45490
   - [Sync] m7: 45490
   - [Target] y: 45490
데이터 동기화 완료! (N=45,490)
------------------------------
최적 비율 발견 (Local Max AUC: 0.773603)
   m1 (Base): 33.07%
   m2 (AUC) : 32.97%
   m7 (DFE) : 33.97%
------------------------------


In [3]:
# 1. 비율 설정
w_opt = [0.3307, 0.3297, 0.3396]  # 최적 비율 (약 1:1:1)
w_safe = [0.20, 0.50, 0.30]       # 안전 비율 (2:5:3)

# 2. 랭크 앙상블 계산
rank_opt = (r1 * w_opt[0]) + (r2 * w_opt[1]) + (r7 * w_opt[2])
rank_safe = (r1 * w_safe[0]) + (r2 * w_safe[1]) + (r7 * w_safe[2])

# 3. AUC 계산
auc_opt = roc_auc_score(y_true_sync, rank_opt)
auc_safe = roc_auc_score(y_true_sync, rank_safe)

print("=" * 40)
print(f"[1:1:1] 최적 비율 AUC : {auc_opt:.6f}")
print(f"[2:5:3] 방어 비율 AUC : {auc_safe:.6f}")
print("-" * 40)
diff = auc_opt - auc_safe
print(f"점수 차이 (Gap)      : {diff:.6f}")

if diff > 0.0002:
    print("결론: 2:5:3은 성능 손실이 너무 큽니다. [1:1:1]로 가십시오.")
else:
    print("결론: 차이가 미미합니다. Private 방어를 위해 [2:5:3]도 고려 가능합니다.")
print("=" * 40)

[1:1:1] 최적 비율 AUC : 0.773603
[2:5:3] 방어 비율 AUC : 0.773499
----------------------------------------
점수 차이 (Gap)      : 0.000104
결론: 차이가 미미합니다. Private 방어를 위해 [2:5:3]도 고려 가능합니다.


In [5]:
import pandas as pd
import numpy as np
import os

# 1. 경로 설정
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'

# 2. 기준 파일 로드
sample_sub = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

# 3. 모델 로드 및 정렬
def load_and_align(filename):
    df = pd.read_csv(os.path.join(SUB_PATH, filename))
    if len(df) != len(sample_sub):
        raise ValueError(f"[Error] {filename} 행 개수 불일치")
    if 'index' in df.columns:
         df = df.set_index('index').reindex(sample_sub['index']).reset_index()
    elif 'id' in df.columns:
         df = df.set_index('id').reindex(sample_sub['id']).reset_index()
    return df['voted'].values

print(">>> 데이터 로딩 중...")
s1_voted = load_and_align('0130-1923.csv')               # m1 (Base)
s2_voted = load_and_align('Pure_0781_AUC_0.77322.csv')   # m2 (Private Ace)
s7_voted = load_and_align('m7_Refined_test_preds.csv')   # m7 (Deep FE)

# 4. 방향성 체크
corr = np.corrcoef(s1_voted, s7_voted)[0, 1]
m7_ascending = True
if corr < 0:
    print(f"!!! m7 역방향 감지 ({corr:.4f}) -> 자동 반전 적용 !!!")
    m7_ascending = False

# 5. Golden Trinity Ensemble (20 : 50 : 30)
# "잃을 게 없는 상황"에서 Private 점수 떡상을 노리는 최적의 배팅
print(">>> Golden Ratio (2:5:3) 앙상블 생성 중...")

r1 = pd.Series(s1_voted).rank(pct=True)
r2 = pd.Series(s2_voted).rank(pct=True)
if m7_ascending:
    r7 = pd.Series(s7_voted).rank(pct=True)
else:
    r7 = 1.0 - pd.Series(s7_voted).rank(pct=True)

# 가중치 적용
avg_rank = (r1 * 0.20) + (r2 * 0.50) + (r7 * 0.30)

# 6. Scale Restoration (m1 분포 사용)
m1_values_sorted = np.sort(s1_voted)
rank_indices = avg_rank.rank(method='first').astype(int).values - 1
final_voted = m1_values_sorted[rank_indices]

# 7. 저장
submission = sample_sub.copy()
submission['voted'] = final_voted
output_name = os.path.join(SUB_PATH, '47_GOLDEN_TRINITY_20_50_30.csv')
submission.to_csv(output_name, index=False)

print("-" * 30)
print(f"파일 생성 완료: {output_name}")
print("전략: 2(m1) : 5(m2) : 3(m7)")
print("목표: Private 점수 반등 확인 (잃을 것 없는 배팅)")
print("-" * 30)

>>> 데이터 로딩 중...
!!! m7 역방향 감지 (-0.9910) -> 자동 반전 적용 !!!
>>> Golden Ratio (2:5:3) 앙상블 생성 중...
------------------------------
파일 생성 완료: ./submissions/47_GOLDEN_TRINITY_20_50_30.csv
전략: 2(m1) : 5(m2) : 3(m7)
목표: Private 점수 반등 확인 (잃을 것 없는 배팅)
------------------------------
